First, create the model. This must match the model used in the interactive training notebook.

In [1]:
! export LD_PRELOAD=/usr/lib/aarch64-linux-gnu/libgomp.so.1
# sudo systemctl restart nvargus-daemon
import cv2

In [2]:
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [ ]:
model.load_state_dict(torch.load('road_following_model_RESNET18_200.pth'))

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [ ]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [ ]:
torch.save(model_trt.state_dict(), 'road_following_model_trt.pth')

Load the optimized model by executing the cell below

In [3]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt.pth'))

<All keys matched successfully>

Create the racecar class

In [4]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Create the camera class.

In [5]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224)

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [ ]:
STEERING_GAIN = 0.75
STEERING_BIAS = -0.12

car.steering = 1 * STEERING_GAIN + STEERING_BIAS
car.throttle = 0.12

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets


#camera = Camera.instance(width=224, height=224)
#image = widgets.Image(format='jpeg', width=224, height=224)
gain_slider = ipywidgets.FloatSlider(description='gain', min=0.45, max=2,step = 0.05, value = .118, orientation='horizontal')
speed_slider = ipywidgets.FloatSlider(description='speed', min=0.11, max=.16, step=0.01, value = .75, orientation='vertical')
gain_text = ipywidgets.FloatText(description='Gain')
gain_text.value =3.5
#camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)


def on_value_change(change):
    CAR_TRHOTTLE = change.new
    print(CAR_TRHOTTLE)

speed_slider.observe(on_value_change, names='value')

display(ipywidgets.VBox([ipywidgets.HBox([gain_slider,gain_text]), speed_slider]))

In [ ]:
from utils import preprocess
import numpy as np

STEERING_GAIN = -0.75
STEERING_BIAS = -0.12
SPEED_GAIN = .2
car.throttle = 0.15

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    y = float(output[1])

    car.steering = x * STEERING_GAIN + STEERING_BIAS
    #car.steering = x * -1*(gain_slider.value)+ STEERING_BIAS

      
    
    #print (output[0],output[1],x,y,car.steering)
    if abs(y)<.01:
        print(y)
        car.throttle =  0.0

    #else:
    #print('GO')


-0.0084075927734375
-0.0061187744140625
-0.002532958984375
-0.0062713623046875
-0.0092926025390625
-0.0084686279296875
-0.0066986083984375
-0.0089569091796875
-0.0069122314453125
-0.0044708251953125
-0.0032958984375
-0.0048675537109375
-0.0099945068359375
-0.0074310302734375
-0.0070953369140625
-0.0079193115234375
-0.0075531005859375
-0.0084381103515625
-0.0075531005859375
-0.0086822509765625
-0.0045928955078125
-0.0066986083984375
-0.0076141357421875
-0.0092315673828125
-0.0070343017578125
-0.0056610107421875
-0.0099334716796875
-0.0097198486328125
-0.0044097900390625
-0.0056304931640625
-0.0098419189453125
-0.0065155029296875
-0.0087432861328125
-0.0095977783203125
-0.0083770751953125
-0.0058746337890625
-0.0093536376953125
-0.0099639892578125
-0.0088043212890625
-0.0081024169921875
-0.0077667236328125
-0.0088348388671875
-0.0084991455078125
-0.0093841552734375
-0.0063934326171875
-0.0071868896484375
-0.0077972412109375
-0.0076141357421875
-0.0087738037109375
-0.0062713623046875
-0.0

In [ ]:
camera.running = False
camera.cap.release() 
del camera 